## Predicting Cryptocurrency Prices with Regression Analysis

#### The aim of this project is to gain practical experience in handling a regression problem by building models to predict the price of a cryptocurrency. As cryptocurrency markets are highly volatile and influenced by multiple factors, predicting their prices requires a thoughtful approach to data analysis and model selection. For this project, I’ll explore different feature engineering and pre-processing techniques to optimize model performance. I’ll begin by constructing a linear regression model, examining how well it captures trends, and possibly experimenting with other regression techniques to push predictive accuracy further. Altough I could predict for the future I want to gauge just how well my model predicts. In order to do this I will be prediciting prices for 2019 and comparing with the actual this will give us better insight on the predicitive capabilities of my model!
* Data was taken from yfinance (Yahoo Finance) https://finance.yahoo.com/?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAAFrDXzo9HDhc2-KjBxhuvWVClU3MRvryVRC02rLLZOWfnXMxlQmmBYEXJwIiRNI7AIEVWWQa1nip7Y7lrSygIMiZoGdJY77r3RbaEf5-DNXPebzZCDZFBqyGp5xxeSSEY-ds1ZNyMik_8nkyyL0G21mI6GLOLhX7syGtgmyTO1e1
  

In [8]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [9]:
import yfinance as yf #we will be pulling the info for our historical ETH(Ethereum Crypto) prices from this 
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
ticker = 'ETH-USD'
eth_data = yf.download(ticker, start='2017-01-01', end='2021-01-01') 
# Creating the date ranges that we are intrested in but we will be prediciting 2019 
print(eth_data)

[*********************100%***********************]  1 of 1 completed

Price                       Adj Close       Close        High         Low  \
Ticker                        ETH-USD     ETH-USD     ETH-USD     ETH-USD   
Date                                                                        
2017-11-09 00:00:00+00:00  320.884003  320.884003  329.451996  307.056000   
2017-11-10 00:00:00+00:00  299.252991  299.252991  324.717987  294.541992   
2017-11-11 00:00:00+00:00  314.681000  314.681000  319.453003  298.191986   
2017-11-12 00:00:00+00:00  307.907990  307.907990  319.153015  298.513000   
2017-11-13 00:00:00+00:00  316.716003  316.716003  328.415009  307.024994   
...                               ...         ...         ...         ...   
2020-12-27 00:00:00+00:00  682.642334  682.642334  711.393555  628.334961   
2020-12-28 00:00:00+00:00  730.397339  730.397339  745.877747  683.205811   
2020-12-29 00:00:00+00:00  731.520142  731.520142  737.952881  692.149414   
2020-12-30 00:00:00+00:00  751.618958  751.618958  754.303223  720.988892   

*Now lets filter the data to exclude 2019 prices*

In [15]:
eth_filtered = eth_data[(eth_data.index < '2019-01-01') | (eth_data.index >= '2020-01-01') ]

In [19]:
eth_filtered.isna().sum()

Price      Ticker 
Adj Close  ETH-USD    0
Close      ETH-USD    0
High       ETH-USD    0
Low        ETH-USD    0
Open       ETH-USD    0
Volume     ETH-USD    0
dtype: int64

Just checking to see if there are any null values this also shows us the relative features in our data!

For those who are new to the world of cryptocurrency, let's start with a brief introduction before diving into the specifics. Cryptocurrency, often referred to simply as 'crypto,' are a digital or virtual form of currency that relies on cryptography for security. Unlike traditional currencies issued by governments and regulated by central banks, cryptocurrencies operate on decentralized networks using blockchain technology. Okay but what even is that? This means that a crytpo like "Ethereum" operates on a network of computers (nodes) spread worldwide rather than relying on a single, central authority, like a bank or corporation.  This structure allows for transparency, security, and independence from central authorities.

In addition to serving as digital money, cryptocurrencies have a range of applications that go beyond just buying and selling. For instance, they enable fast, low-cost global payments, provide financial services for those without access to traditional banking, and support a variety of online services through smart contracts. Smart contract you say? What makes it so smart? Smart contracts are a self-executing program or code that runs on a blockchain, automatically enforcing and executing the terms of an agreement when certain conditions are met. With crypto, we see innovations across fields like finance, supply chain management, gaming, and art through NFTs (non-fungible tokens).